In [1]:
import os
import json
import random
import csv
import torch
import pandas as pd
import numpy as np
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
import torch.nn.functional as F
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForMaskedLM,
    get_linear_schedule_with_warmup
)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm

In [2]:
train_path     = '/kaggle/input/training-data/training_data.json'          # QA training data
questions_path = '/kaggle/input/evaluation/questions.json'       # List of questions
passages_path  = '/kaggle/input/evaluation/passages.json'        # Retrieval corpus
model_name     = 'cis-lmu/glot500-base'
output_dir     = './retriever_ckpt'
epochs         = 6
batch_size     = 8
learning_rate  = 2e-5
max_length     = 500
warmup_steps   = 1000
do_mlm         = True
mlm_epochs     = 2
mlm_batch_size = 1
device         = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
class QAPairsDataset(Dataset):
    """
    A simple Dataset that returns (question, passage) pairs.
    Expects a list of dicts: {"question": str, "passage": str}.
    """
    def __init__(self, data): self.data = data
    def __len__(self): return len(self.data)
    def __getitem__(self, idx): return self.data[idx]

In [4]:
class MLMDataset(Dataset):
    """
    Dataset for MLM: returns masked inputs and labels for Masked Language Modeling.
    """
    def __init__(self, texts, tokenizer, max_length=128, mask_prob=0.15):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.mask_prob = mask_prob

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        enc = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            return_special_tokens_mask=True
        )
        input_ids = enc['input_ids']
        labels = input_ids.copy()
        for i in range(len(input_ids)):
            if random.random() < self.mask_prob and enc['special_tokens_mask'][i] == 0:
                input_ids[i] = self.tokenizer.mask_token_id
            else:
                # Use -100 to signal CrossEntropyLoss.ignore_index, so that
                # unmasked positions are ignored in the MLM loss computation.
                labels[i] = -100
        return {
            'input_ids': torch.tensor(input_ids),
            'labels': torch.tensor(labels)
        }

In [5]:
def collate_qa(batch, tokenizer, max_length=128):
    """
    Collate function for question-passage pairs.

    Tokenizes and pads questions and passages in the batch.
    
    """
    questions = [ex['question'] for ex in batch]
    passages = [ex['passage'] for ex in batch]
    q_enc = tokenizer(questions, padding=True, truncation=True,
                      max_length=max_length, return_tensors='pt')
    p_enc = tokenizer(passages, padding=True, truncation=True,
                      max_length=max_length, return_tensors='pt')
    return {
        'q_input_ids': q_enc.input_ids,
        'q_attention_mask': q_enc.attention_mask,
        'p_input_ids': p_enc.input_ids,
        'p_attention_mask': p_enc.attention_mask
    }


def collate_mlm(batch):
    """
    Collate function for MLM.

    Pads input_ids and labels to the same length and creates attention_mask.
    
    """
    input_ids = [ex['input_ids'] for ex in batch]
    labels = [ex['labels'] for ex in batch]
    input_ids = nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=0)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)
    attention_mask = (input_ids != 0).long()
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}


def mean_pooling(token_embeddings, attention_mask):
    """
    Mean-pool token embeddings, masking out padding tokens.
    """
    mask = attention_mask.unsqueeze(-1).float()
    summed = torch.sum(token_embeddings * mask, dim=1)
    counts = torch.clamp(mask.sum(dim=1), min=1e-9)
    return summed / counts


In [6]:
class BiEncoderRetriever(nn.Module):
    """
    Bi-encoder model using a shared transformer for both questions and passages.
    """
    def __init__(self, model_name="cis-lmu/glot500-base"):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(model_name)

    def forward(self, input_ids, attention_mask):
        outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=True
        )
        return mean_pooling(outputs.last_hidden_state, attention_mask)

In [7]:
def info_nce_loss(q_embeds, p_embeds, temperature=0.05):
    """
    Compute InfoNCE contrastive loss between question and passage embeddings.

    Args:
        q_embeds (Tensor): Question embeddings [B, D].
        p_embeds (Tensor): Passage embeddings [B, D].
        temperature (float): Scaling factor for logits.

    """
    q_norm = nn.functional.normalize(q_embeds, dim=1)
    p_norm = nn.functional.normalize(p_embeds, dim=1)
    sims = torch.matmul(q_norm, p_norm.t()) / temperature
    labels = torch.arange(sims.size(0), device=sims.device)
    return nn.CrossEntropyLoss()(sims, labels)

In [8]:
def load_json(path):
    with open(path,'r',encoding='utf-8-sig') as f: return json.load(f)


def load_qa_data(path):
    df = pd.read_json(path)
    return df.to_dict(orient='records')

def extract_passages(examples): return [ex['passage'] for ex in examples]

In [9]:
def train(
    train_path, model_name, output_dir,
    epochs, batch_size, learning_rate,
    max_length, warmup_steps, device,
    do_mlm, mlm_epochs , mlm_batch_size
):
    """
    Execute optional MLM pre-training and contrastive retriever finetuning.

    Args:
        train_path (str): JSONL file path for QA data.
        model_name (str): HF model ID for encoder.
        output_dir (str): Directory to save checkpoints.
        epochs (int): Number of contrastive training epochs.
        batch_size (int): Training batch size.
        learning_rate (float): Optimizer learning rate.
        max_length (int): Maximum sequence length.
        warmup_steps (int): Scheduler warmup steps.
        device (str or None): 'cuda' or 'cpu'.
        do_mlm (bool): Whether to run MLM pre-training.
        mlm_epochs (int): Number of MLM epochs.

    Returns:
        None
    """

    device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    qa_data = load_json(train_path)

    # MLM phase
    if do_mlm:
        passages = [ex['passage'] for ex in qa_data]
        mlm_dataset = MLMDataset(passages, tokenizer, max_length=max_length)
        mlm_loader = DataLoader(mlm_dataset, batch_size=mlm_batch_size,
                                shuffle=True, collate_fn=collate_mlm)
        mlm_model = AutoModelForMaskedLM.from_pretrained(model_name).to(device)
        mlm_optimizer = AdamW(mlm_model.parameters(), lr=learning_rate)
        total_mlm_steps = len(mlm_loader) * mlm_epochs
        mlm_scheduler = get_linear_schedule_with_warmup(
            mlm_optimizer, num_warmup_steps=warmup_steps,
            num_training_steps=total_mlm_steps
        )
        mlm_model.train()
        for epoch in range(1, mlm_epochs+1):
            mlm_loss_accum = 0.0
            for batch in tqdm(mlm_loader, desc=f"MLM Epoch {epoch}/{mlm_epochs}"):
                mlm_optimizer.zero_grad()
                out = mlm_model(
                    input_ids=batch['input_ids'].to(device),
                    attention_mask=batch['attention_mask'].to(device),
                    labels=batch['labels'].to(device)
                )
                loss = out.loss
                loss.backward()
                mlm_optimizer.step()
                mlm_scheduler.step()
                mlm_loss_accum += loss.item()
            print(f"MLM Epoch {epoch}/{mlm_epochs} — Loss: {mlm_loss_accum/len(mlm_loader):.4f}")
        mlm_ckpt = os.path.join(output_dir, 'mlm_finetuned')
        os.makedirs(mlm_ckpt, exist_ok=True)
        mlm_model.save_pretrained(mlm_ckpt)
        tokenizer.save_pretrained(mlm_ckpt)

        # FREEING UP GPU MEMORY
        del mlm_model
        del mlm_optimizer
        del mlm_scheduler
        del mlm_loader
        torch.cuda.empty_cache()
    
    # Contrastive QA phase
    qa_loader = DataLoader(
        QAPairsDataset(qa_data), batch_size=batch_size, shuffle=True,
        collate_fn=lambda b: collate_qa(b, tokenizer, max_length)
    )
    retriever = BiEncoderRetriever(model_name).to(device)
    optimizer = AdamW(retriever.parameters(), lr=learning_rate)
    total_steps = len(qa_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )
    retriever.train()
    for epoch in range(1, epochs+1):
        loss_accum = 0.0
        for batch in tqdm(qa_loader, desc=f"Contrast Epoch {epoch}/{epochs}"):
            optimizer.zero_grad()
            q_emb = retriever(batch['q_input_ids'].to(device), batch['q_attention_mask'].to(device))
            p_emb = retriever(batch['p_input_ids'].to(device), batch['p_attention_mask'].to(device))
            loss = info_nce_loss(q_emb, p_emb)
            loss.backward()
            optimizer.step()
            scheduler.step()
            loss_accum += loss.item()
        print(f"Contrastive Epoch {epoch}/{epochs} — Loss: {loss_accum/len(qa_loader):.4f}")
        ckpt_dir = os.path.join(output_dir, f"epoch_{epoch}")
        os.makedirs(ckpt_dir, exist_ok=True)
        retriever.encoder.save_pretrained(ckpt_dir)
        tokenizer.save_pretrained(ckpt_dir)

    print("Training complete.")

In [10]:

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'   # INFO=0, WARNING=1, ERROR=2, FATAL=3
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)


In [11]:
train(train_path, model_name, output_dir, 
      epochs, batch_size, learning_rate, 
      max_length, warmup_steps, device,
      do_mlm, mlm_epochs , mlm_batch_size)

tokenizer_config.json:   0%|          | 0.00/496 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/7.66M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

2025-08-05 00:09:26.576524: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754352566.948548      18 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754352567.054037      18 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

Contrastive Epoch 4/6 — Loss: 0.0915


Contrast Epoch 5/6:   0%|          | 0/1143 [00:00<?, ?it/s]

Contrastive Epoch 5/6 — Loss: 0.0674


Contrast Epoch 6/6:   0%|          | 0/1143 [00:00<?, ?it/s]

Contrastive Epoch 6/6 — Loss: 0.0573
Training complete.


In [12]:
questions = load_json(questions_path)
passages  = load_json(passages_path)

In [13]:
def retrieve_embeddings(model, tokenizer, texts, batch_size=16):
    embs=[]
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        enc = tokenizer(batch, padding=True, truncation=True,
                        max_length=max_length, return_tensors='pt')
        for k in enc: enc[k] = enc[k].to(device)
        with torch.no_grad(): out = model(**enc)
        embs.append(mean_pooling(out.last_hidden_state, enc['attention_mask']).cpu())
    return torch.cat(embs, 0)

In [14]:
tok    = AutoTokenizer.from_pretrained(model_name)
mod_zs = AutoModel.from_pretrained(model_name).to(device)
pass_emb_zs = retrieve_embeddings(mod_zs, tok, passages, batch_size)
mod_ft      = AutoModel.from_pretrained(os.path.join(output_dir, 'epoch_' + str(epochs))).to(device)
pass_emb_ft = retrieve_embeddings(mod_ft, tok, passages, batch_size)

Some weights of XLMRobertaModel were not initialized from the model checkpoint at cis-lmu/glot500-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# 3) Retrieve top-k function

def retrieve_topk(q_emb, pass_embs, texts, k=3):
    sims = cosine_similarity(q_emb, pass_embs)
    topk_idx = sims.argsort(axis=1)[:, -k:][:, ::-1][0]
    return [texts[i] for i in topk_idx]

In [16]:
# Run retrieval
zero_shot = []
finetuned = []
for q in questions:
    enc_q = tok(q,return_tensors='pt',truncation=True,max_length=max_length)
    for k in enc_q: enc_q[k]=enc_q[k].to(device)
    with torch.no_grad(): 
        q_emb=mean_pooling(mod_zs(**enc_q).last_hidden_state, enc_q['attention_mask']).cpu()
    zero_shot.append(retrieve_topk(q_emb, pass_emb_zs, passages))
    with torch.no_grad(): 
        q_emb_ft=mean_pooling(mod_ft(**enc_q).last_hidden_state, enc_q['attention_mask']).cpu()
    finetuned.append(retrieve_topk(q_emb_ft, pass_emb_ft, passages))

In [17]:
# -----------------------
# TF-IDF Baseline
# -----------------------
vectorizer = TfidfVectorizer(max_features=50000)
corpus_tfidf = vectorizer.fit_transform(passages)
q_tfidf = vectorizer.transform(questions)
sims_tfidf = cosine_similarity(q_tfidf, corpus_tfidf)
top3_tfidf = sims_tfidf.argsort(axis=1)[:, -3:][:, ::-1]

In [18]:
# -----------------------
# Save Results to CSV
# -----------------------
csv_path = 'retrieval_results.csv'
with open(csv_path, 'w', newline='', encoding='utf-8-sig') as csvfile:
    writer = csv.writer(csvfile)
    header = ['question',
              'tfidf_1','tfidf_2','tfidf_3',
              'zero1','zero2','zero3',
              'ft1','ft2','ft3']
    writer.writerow(header)
    for i, q in enumerate(questions):
        row = [q]
        # TF-IDF passages
        for idx in top3_tfidf[i]:
            row.append(passages[idx])
        # zero-shot passages
        row.extend(zero_shot[i])
        # fine-tuned passages
        row.extend(finetuned[i])
        writer.writerow(row)
print(f"Saved retrieval results to {csv_path}")

Saved retrieval results to retrieval_results.csv


# Duplicate Detection via FAISS using the fine-tuned model

In [19]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
!pip install faiss-cpu -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 38.3 MB/s eta 0:00:00


In [20]:
import faiss

# Build index for passage embeddings
pass_emb_ft_np = pass_emb_ft.cpu().detach().numpy().astype('float32')
faiss.normalize_L2(pass_emb_ft_np)
dim = pass_emb_ft_np.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(pass_emb_ft_np)


# Search each passage's nearest neighbor (k=2 includes itself)
D, I = index.search(pass_emb_ft_np, 2)
duplicates = []
threshold = 0.90
for i in range(len(passages)):
    neighbor = I[i,1]
    score = float(D[i,1])
    if neighbor != i and score > threshold:
        duplicates.append((i, neighbor, score))

# Save duplicates to CSV
dup_csv = 'duplicates.csv'
with open(dup_csv, 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f)
    writer.writerow(['idx1','idx2','score','text1','text2'])
    for i,j,score in duplicates:
        writer.writerow([i, j, f'{score:.4f}', passages[i], passages[j]])
print(f"Saved duplicates to {dup_csv}")

Saved duplicates to duplicates.csv
